# 深層学習体験会: テキストから動画生成（LTX-Video）

このノートブックでは、**LTX-Video**モデルを使って、テキストの説明文だけから動画を生成する方法を学びます。

## テキストから動画生成とは？

**Text-to-Video**は、テキストの説明文を入力するだけで、それに基づいた動画を自動生成する技術です。画像生成（Text-to-Image）の動画版と言えます。

### LTX-Videoの特徴
- **完全なテキスト制御**: プロンプトだけで動画を生成
- **高品質出力**: 滑らかで自然な動画
- **柔軟な設定**: 解像度、フレーム数、スタイルを調整可能

### 応用例
- 映画の絵コンテ作成
- 広告動画の試作
- ストーリーボードの自動生成
- SNS用のショート動画作成
- 教育コンテンツの視覚化

## 重要な注意事項
- **GPU要件**: 無料のT4 GPUで実行可能ですが、解像度とフレーム数によっては制限があります
  - デフォルト設定（832x480、73フレーム）は問題なく動作
  - 768x512で121フレームの場合、デコード時にメモリ不足になる可能性があります
- **より高速・高品質な生成には上位GPUを推奨**
- **フレーム数の調整**: n フレームの動画を生成したい場合は、frames を n+1 に設定してください
  - 例: 72フレームの動画を作りたい → frames = 73 に設定
- **プロンプトの重要性**: 詳細で具体的なプロンプトを使用すると、より良い結果が得られます
- **FPS**: 動画は24fpsで生成されます

## このノートブックの流れ
1. 環境の準備とモデルのダウンロード
2. プロンプトとパラメータの設定
3. 動画の生成と再生

In [ ]:
# @title 環境の準備
# --- PyTorchのインストール ---
# PyTorch 2.6.0とtorchvision 0.21.0をインストール（深層学習フレームワーク）
!pip install torch==2.6.0 torchvision==0.21.0

# /contentディレクトリに移動
%cd /content

# --- 設定変数 ---
# 推論用にモデルを常にロードするかどうか
Always_Load_Models_for_Inference = False
# T5XXLの16bit版を使用するかどうか（Falseの場合は8bit量子化版を使用）
Use_t5xxl_fp16 = False

# --- 依存ライブラリのインストール ---
# torchsde: 確率微分方程式ソルバー（ノイズ除去に使用）
# einops: テンソル操作を簡潔に記述するライブラリ
# diffusers: 拡散モデルのライブラリ
# accelerate: モデルの高速化とメモリ最適化
# xformers: 高速なアテンション機構
# av: 動画ファイルの読み書き用ライブラリ
!pip install -q torchsde einops diffusers accelerate xformers==0.0.29.post2
!pip install av

# --- ComfyUIのクローンとセットアップ ---
# ComfyUI: 動画生成のバックエンドフレームワーク
!git clone https://github.com/Isi-dev/ComfyUI
%cd /content/ComfyUI

# aria2（高速ダウンローダー）とffmpeg（動画処理）をインストール
!apt -y install -qq aria2 ffmpeg

# --- 必要なモデルファイルをダウンロード ---
# LTX-Videoメインモデル（約2Bパラメータ、動画生成の中核）
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/ltx-video-2b-v0.9.5.safetensors -d /content/ComfyUI/models/checkpoints -o ltx-video-2b-v0.9.5.safetensors

# テキストエンコーダー（T5-XXL）をダウンロード
# 条件分岐: fp16（16bit）版またはfp8（8bit量子化）版を選択
if Use_t5xxl_fp16:
    # 16bit版（高精度だがメモリ消費大）
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp16.safetensors
else:
    # 8bit量子化版（省メモリで推奨）
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp8_e4m3fn_scaled.safetensors

# --- 必要なライブラリをインポート ---
import torch  # PyTorchフレームワーク
import numpy as np  # 数値計算用
from PIL import Image  # 画像処理用
import gc  # ガベージコレクション（メモリ管理）
import sys  # システム関連の操作
import random  # 乱数生成
import os  # ファイル・ディレクトリ操作
import imageio  # 動画ファイルの読み書き
from google.colab import files  # Google Colabのファイルアップロード機能
from IPython.display import display, HTML  # Jupyter Notebookでの表示用

# ComfyUIのモジュールをインポートできるようにパスを追加
sys.path.insert(0, '/content/ComfyUI')

# --- ComfyUIのモジュールをインポート ---
from comfy import model_management  # モデルのメモリ管理

# 基本的なノード（処理単位）をインポート
from nodes import (
    CheckpointLoaderSimple,  # モデルの読み込み
    CLIPLoader,  # テキストエンコーダーの読み込み
    CLIPTextEncode,  # テキストのエンコード（埋め込みベクトル化）
    VAEDecode  # 潜在表現から画像/動画へのデコード
)

# カスタムサンプラー関連のノード
from comfy_extras.nodes_custom_sampler import (
    KSamplerSelect,  # サンプリング手法の選択
    SamplerCustom  # カスタムサンプラーの実行
)

# LTX-Video専用のノード
from comfy_extras.nodes_lt import (
    LTXVConditioning,  # LTX-Video用の条件付け
    LTXVScheduler,  # ノイズスケジューラー
    EmptyLTXVLatentVideo  # 空の潜在動画（ノイズから開始）
)

# --- 各ノードのインスタンスを作成 ---
# これらのオブジェクトを通じて各処理を実行する
checkpoint_loader = CheckpointLoaderSimple()  # モデル読み込み用
clip_loader = CLIPLoader()  # テキストエンコーダー読み込み用
clip_encode_positive = CLIPTextEncode()  # ポジティブプロンプトのエンコード用
clip_encode_negative = CLIPTextEncode()  # ネガティブプロンプトのエンコード用
scheduler = LTXVScheduler()  # ノイズ除去スケジュール管理用
sampler_select = KSamplerSelect()  # サンプラー選択用
conditioning = LTXVConditioning()  # 条件付け処理用
empty_latent_video = EmptyLTXVLatentVideo()  # 空の潜在動画生成用
sampler = SamplerCustom()  # サンプリング実行用
vae_decode = VAEDecode()  # VAEデコード用


def clear_memory():
    """
    GPU（VRAM）とCPU RAMのメモリを解放する関数
    
    動画生成は大量のメモリを消費するため、処理の合間に
    不要なオブジェクトを削除してメモリを確保する必要があります。
    """
    # Pythonのガベージコレクションを実行（不要なオブジェクトを削除）
    gc.collect()
    
    # CUDAが利用可能な場合（GPUがある場合）
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # GPUキャッシュをクリア
        torch.cuda.ipc_collect()  # プロセス間通信メモリをクリア
    
    # グローバル変数に残っているテンソルを削除
    for obj in list(globals().values()):
        if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
            del obj
    
    # 再度ガベージコレクションを実行
    gc.collect()


def generate_video(
    positive_prompt: str = "A drone quickly rises through a bank of morning fog...",  # 生成したい動画の説明（ポジティブプロンプト）
    negative_prompt: str = "low quality, worst quality...",  # 避けたい要素（ネガティブプロンプト）
    width: int = 768,  # 動画の幅（ピクセル、32の倍数である必要）
    height: int = 512,  # 動画の高さ（ピクセル、32の倍数である必要）
    seed: int = 0,  # ランダムシード（0の場合はランダム、それ以外は再現可能）
    steps: int = 30,  # 生成ステップ数（多いほど高品質だが時間がかかる）
    cfg_scale: float = 2.05,  # CFGスケール（プロンプトへの忠実度、大きいほど忠実）
    sampler_name: str = "res_multistep",  # サンプリング手法の名前
    length: int = 49,  # 生成するフレーム数（n+1フレームでn秒の動画）
    fps: int = 24  # フレームレート（1秒あたりのフレーム数）
):
    """
    テキストプロンプトから動画を生成するメイン関数
    
    引数:
        positive_prompt: 生成したい動画の内容を詳細に記述したテキスト
        negative_prompt: 避けたい要素を記述したテキスト
        width: 動画の幅（32の倍数）
        height: 動画の高さ（32の倍数）
        seed: ランダムシード（0=ランダム、それ以外=再現可能）
        steps: ノイズ除去のステップ数
        cfg_scale: Classifier-Free Guidanceのスケール（プロンプトへの忠実度）
        sampler_name: 使用するサンプリングアルゴリズム名
        length: 生成するフレーム数
        fps: 動画のフレームレート（通常24fps）
    """
    
    # 推論モード（学習しない）で実行
    with torch.inference_mode():
        print("テキストエンコーダーを読み込み中...")
        # CLIPテキストエンコーダーを読み込む
        # テキストプロンプトを数値ベクトルに変換するために使用
        clip = clip_loader.load_clip("t5xxl_fp8_e4m3fn_scaled.safetensors", "ltxv", "default")[0]
        print("テキストエンコーダーを読み込みました！")

    try:
        # --- 入力の検証 ---
        # 幅と高さが32の倍数であることを確認（モデルの制約）
        assert width % 32 == 0, "幅は32の倍数である必要があります"
        assert height % 32 == 0, "高さは32の倍数である必要があります"

        # --- プロンプトのエンコード ---
        # ポジティブプロンプト（生成したい内容）を埋め込みベクトルに変換
        positive = clip_encode_positive.encode(clip, positive_prompt)[0]
        # ネガティブプロンプト（避けたい内容）を埋め込みベクトルに変換
        negative = clip_encode_negative.encode(clip, negative_prompt)[0]

        # テキストエンコーダーはもう不要なのでメモリから削除
        del clip
        torch.cuda.empty_cache()
        gc.collect()
        print("テキストエンコーダーをメモリから削除しました")

        # --- 空の潜在動画を生成 ---
        # ランダムノイズから開始するための初期状態を作成
        empty_latent = empty_latent_video.generate(width, height, length)[0]

        # --- ノイズ除去スケジュールを設定 ---
        # sigmas: ノイズ除去の各ステップでのノイズレベルを定義
        # 引数: (ステップ数, CFGスケール, denoise強度, シフト有効, シフトベース)
        sigmas = scheduler.get_sigmas(steps, cfg_scale, 0.95, True, 0.1)[0]
        
        # サンプリング手法を選択（res_multistep, euler, dpmpp_2mなど）
        selected_sampler = sampler_select.get_sampler(sampler_name)[0]
        
        # 条件付けを結合（ポジティブとネガティブのプロンプトを統合）
        # 25.0: 動画の長さに関する条件付け強度
        conditioned = conditioning.append(positive, negative, 25.0)

        # --- モデルとVAEの読み込み ---
        print("モデルとVAEを読み込み中...")
        # LTX-Videoのメインモデルを読み込む
        # model: ノイズ除去モデル（潜在空間で動作）
        # vae: 変分オートエンコーダー（潜在空間⇔ピクセル空間の変換）
        model, _, vae = checkpoint_loader.load_checkpoint("ltx-video-2b-v0.9.5.safetensors")
        print("モデルとVAEを読み込みました！")

        # --- 動画の生成（サンプリング） ---
        print("動画を生成中...")
        # ノイズ除去プロセスを実行して潜在表現を生成
        sampled = sampler.sample(
            model=model,  # 使用するモデル
            add_noise=True,  # ノイズを追加するか
            noise_seed=seed if seed != 0 else random.randint(0, 2**32),  # シード値（0の場合はランダム）
            cfg=cfg_scale,  # CFGスケール（プロンプトへの忠実度）
            positive=conditioned[0],  # ポジティブ条件
            negative=conditioned[1],  # ネガティブ条件
            sampler=selected_sampler,  # サンプリングアルゴリズム
            sigmas=sigmas,  # ノイズ除去スケジュール
            latent_image=empty_latent  # 初期潜在表現（ノイズ）
        )[0]

        # モデルはもう不要なのでメモリから削除
        del model
        torch.cuda.empty_cache()
        gc.collect()
        print("モデルをメモリから削除しました")

        # --- VAEデコード（潜在表現→ピクセル） ---
        # 勾配計算なしで実行（推論のみ）
        with torch.no_grad():
            try:
                print("潜在表現をデコード中...")
                # VAEで潜在表現を実際の画像フレームにデコード
                # sampled: 潜在空間での動画表現 → decoded: ピクセル空間での動画フレーム
                decoded = vae_decode.decode(vae, sampled)[0].detach()
                print("潜在表現をデコードしました！")
                
                # VAEはもう不要なのでメモリから削除
                del vae
                torch.cuda.empty_cache()
                gc.collect()
                print("VAEをメモリから削除しました")

                # --- MP4ファイルとして保存 ---
                output_path = "/content/output.mp4"
                # imageioのライターで動画ファイルを作成
                with imageio.get_writer(output_path, fps=fps) as writer:
                    # 各フレームを処理して書き込み
                    for i, frame in enumerate(decoded):
                        # フレームをCPUに転送してNumPy配列に変換
                        # [0-1]の範囲の浮動小数点数を[0-255]の整数に変換
                        frame_np = (frame.cpu().numpy() * 255).astype(np.uint8)
                        # フレームを動画ファイルに追加
                        writer.append_data(frame_np)
                        # 定期的にメモリをクリア（10フレームごと）
                        if i % 10 == 0:
                            torch.cuda.empty_cache()

                print(f"正常に{len(decoded)}フレームを処理しました")

            except Exception as e:
                # デコード中にエラーが発生した場合
                print(f"デコードエラー: {str(e)}")
                raise

        # --- 動画の表示 ---
        print("動画を表示中...")
        display_video(output_path)

    except Exception as e:
        # 動画生成に失敗した場合
        print(f"動画生成に失敗しました: {str(e)}")
        raise
    finally:
        # 成功・失敗に関わらず、最後にメモリをクリア
        clear_memory()


def display_video(video_path):
    """
    生成された動画をJupyter Notebook内で表示する関数
    
    引数:
        video_path: 表示する動画ファイルのパス
    """
    from IPython.display import HTML
    from base64 import b64encode

    # 動画ファイルをバイナリモードで読み込み
    mp4 = open(video_path,'rb').read()
    # Base64エンコードしてHTMLに埋め込める形式に変換
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # HTMLのvideoタグで動画を表示（自動再生、ループ、コントロール付き）
    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))

# セットアップ完了メッセージ
print("✅ 環境のセットアップが完了しました！")

## ステップ1: 環境の準備

このセルでは、以下の作業を行います:

### インストール
1. **PyTorch 2.6.0**: 深層学習フレームワーク
2. **Diffusers**: 拡散モデルライブラリ
3. **Accelerate**: モデルの高速化
4. **ComfyUI**: 動画生成のバックエンド
5. **av**: 動画ファイル処理

### モデルのダウンロード
- **ltx-video-2b-v0.9.5.safetensors**: メインの動画生成モデル（約2GBのパラメータ）
- **t5xxl_fp8_e4m3fn_scaled.safetensors**: テキストエンコーダー（軽量版、8bit量子化）

### 関数の定義
- `generate_video()`: テキストプロンプトから動画を生成
- `display_video()`: 生成された動画をノートブック内で表示
- `clear_memory()`: GPU/CPUメモリを解放

実行には5〜10分かかる場合があります。「Environment Setup Complete!」と表示されれば準備完了です。

## ステップ2: 動画生成パラメータの設定と実行

### 調整可能なパラメータ

#### プロンプト設定
- **positive_prompt**: 生成したい動画の内容を詳細に記述
  - **重要**: できるだけ詳細で具体的に書く
  - 例: "A drone quickly rises through a bank of morning fog, revealing a pristine alpine lake..."
  - カメラワーク、被写体、照明、雰囲気などを含めると良い
- **negative_prompt**: 避けたい要素を指定
  - 低品質、歪み、動きのブレ、解剖学的な誤りなど

#### 解像度設定
- **width**: 動画の幅（32の倍数である必要があります）
- **height**: 動画の高さ（32の倍数である必要があります）
- **推奨設定**:
  - 832x480（ワイドスクリーン、デフォルト）
  - 768x512（バランス型）
  - 512x768（縦長）

#### 生成パラメータ
- **seed**: ランダムシード
  - 0 = ランダム
  - 特定の値 = 同じ結果を再現可能
- **steps**: 生成ステップ数（15〜30を推奨）
  - 多いほど高品質だが時間がかかる
- **cfg_scale**: プロンプトへの忠実度（1.5〜3.0を推奨）
  - 大きいほどプロンプトに忠実
- **sampler_name**: サンプリング方法
  - **res_multistep**（推奨、デフォルト）
  - euler, dpmpp_2m, ddim, lms なども選択可能

#### 動画設定
- **frames**: 生成するフレーム数（n+1フレームを設定してn秒の動画を生成）
  - 例: 73フレーム = 約3秒（24fps）
  - 注意: 多すぎるとメモリ不足になる可能性
- **fps**: フレームレート（通常は24fps）

### プロンプトのコツ
1. **カメラワークを明示**: "drone rises", "camera glides forward", "tracking shot"
2. **照明を記述**: "golden light of sunrise", "soft moonlight", "harsh shadows"
3. **動きを具体的に**: "slowly moving", "quickly rises", "gently drifting"
4. **雰囲気を追加**: "dreamy atmosphere", "tranquil", "dramatic"
5. **詳細を含める**: 色、質感、天候、時間帯など

### 実行方法
1. パラメータを調整
2. セルを実行
3. 動画生成が開始されます（数分かかります）
4. 完了すると、動画が自動的に表示されます

**ヒント**: 最初は小さいフレーム数（25〜49）と低解像度で試して、結果を確認してから調整することをお勧めします。

In [ ]:
# @title 動画生成パラメータ

# --- プロンプト設定 ---
# 生成したい動画の内容を詳細に記述（英語推奨）
# カメラワーク、被写体、照明、雰囲気などを具体的に記述すると良い結果が得られます
positive_prompt = "A drone quickly rises through a bank of morning fog, revealing a pristine alpine lake surrounded by snow-capped mountains. The camera glides forward over the glassy water, capturing perfect reflections of the peaks. As it continues, the perspective shifts to reveal a lone wooden cabin with a curl of smoke from its chimney, nestled among tall pines at the lake's edge. The final shot tracks upward rapidly, transitioning from intimate to epic as the full mountain range comes into view, bathed in the golden light of sunrise breaking through scattered clouds." # @param {"type":"string"}

# 避けたい要素を記述（品質低下、歪み、不自然な動きなど）
negative_prompt = "low quality, worst quality, deformed, distorted, disfigured, motion smear, motion artifacts, fused fingers, bad anatomy, weird hand, ugly" # @param {"type":"string"}

# --- 解像度設定 ---
# 動画の幅（ピクセル、32の倍数である必要があります）
# 推奨値: 768, 832, 512など
width = 832 # @param {"type":"number"}

# 動画の高さ（ピクセル、32の倍数である必要があります）
# 推奨値: 480, 512, 768など
height = 480 # @param {"type":"number"}

# --- 生成パラメータ ---
# ランダムシード（0の場合はランダム、それ以外は再現可能な結果）
seed = 0 # @param {"type":"integer"}

# 生成ステップ数（15〜30を推奨）
# 多いほど高品質ですが、時間がかかります
steps = 25 # @param {"type":"integer", "min":1, "max":100}

# CFGスケール（Classifier-Free Guidanceのスケール）
# プロンプトへの忠実度を制御（1.5〜3.0を推奨）
# 大きいほどプロンプトに忠実になりますが、過度に大きいと不自然になります
cfg_scale = 2.05 # @param {"type":"number", "min":1, "max":20}

# サンプリング手法の選択
# res_multistep: 高品質（推奨）
# euler: 標準的
# dpmpp_2m: 高速
# ddim, lms: その他の手法
sampler_name = "res_multistep" # @param ["res_multistep", "euler", "dpmpp_2m", "ddim", "lms"]

# --- 動画設定 ---
# 生成するフレーム数
# 注意: n秒の動画を作りたい場合は n+1 に設定してください
# 例: 73フレーム = 約3秒（24fps）、49フレーム = 約2秒
# フレーム数が多いほどメモリを消費します
frames = 73 # @param {"type":"integer", "min":1, "max":120}

# フレームレート（1秒あたりのフレーム数）
# 24fps: 映画品質（推奨）
# 30fps: 標準的な動画
fps = 24 # @param {"type":"integer", "min":1, "max":60}

# --- 動画生成の実行 ---
# 推論モード（学習しない）で実行
with torch.inference_mode():
    # 設定したパラメータで動画を生成
    generate_video(
        positive_prompt=positive_prompt,  # ポジティブプロンプト
        negative_prompt=negative_prompt,  # ネガティブプロンプト
        width=width,  # 動画の幅
        height=height,  # 動画の高さ
        seed=seed,  # ランダムシード
        steps=steps,  # 生成ステップ数
        cfg_scale=cfg_scale,  # CFGスケール
        sampler_name=sampler_name,  # サンプリング手法
        length=frames,  # フレーム数
        fps=fps  # フレームレート
    )

# 生成完了後、メモリをクリア
clear_memory()